In [1]:
import logfire
import polars as pl
from logfire.experimental.query_client import AsyncLogfireQueryClient
from loguru import logger
from pydantic_ai import Agent

logfire.configure()
logger.configure(handlers=[logfire.loguru_handler()])

%load_ext autoreload
%autoreload 2

Logfire project URL: ]8;id=984060;https://logfire.pydantic.dev/HamzaFarhan/knd\https://logfire.pydantic.dev/HamzaFarhan/knd]8;;\


In [43]:
agent = Agent(model="google-gla:gemini-1.5-flash", name="knd_evals")


@agent.result_validator
def validate_result(result: str) -> str:
    if "yoo" not in result:
        logfire.error("Result does not contain 'yoo'", _tags=["yoo_check"])
    return result


In [44]:
res = await agent.run(user_prompt="Hello, how are you?")

09:51:06.356 knd_evals run prompt=Hello, how are you?
09:51:06.357   preparing model and tools run_step=1
09:51:06.358   model request
09:51:08.149   handle model response


In [45]:
res.all_messages()

[ModelRequest(parts=[UserPromptPart(content='Hello, how are you?', timestamp=datetime.datetime(2025, 1, 21, 9, 51, 6, 357821, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[TextPart(content='I am doing well, thank you for asking!  How are you today?\n', part_kind='text')], timestamp=datetime.datetime(2025, 1, 21, 9, 51, 8, 147626, tzinfo=datetime.timezone.utc), kind='response')]

In [48]:
query = """
WITH agent_traces AS (
  SELECT DISTINCT trace_id 
  FROM records 
  WHERE attributes->>'agent_name' = 'knd_evals'
)
SELECT 
  r.trace_id,
  r.span_id,
  r.span_name,
  r.start_timestamp,
  r.end_timestamp,
  r.duration,
  r.level,
  r.message,
  r.tags,
  r.attributes->>'agent_name' as agent_name
FROM records r
JOIN agent_traces at ON r.trace_id = at.trace_id
ORDER BY r.trace_id, r.start_timestamp;
"""

async with AsyncLogfireQueryClient(read_token="H0CTvcy0WCrl6xjxm8r8ZjWxP3LPSq5Mzdv81GvXXRPz") as client:
    df_from_arrow = pl.from_arrow(await client.query_arrow(sql=query))
    print(df_from_arrow)


shape: (20, 10)
┌────────────┬────────────┬────────────┬───────────┬───┬───────┬───────────┬───────────┬───────────┐
│ trace_id   ┆ span_id    ┆ span_name  ┆ start_tim ┆ … ┆ level ┆ message   ┆ tags      ┆ agent_nam │
│ ---        ┆ ---        ┆ ---        ┆ estamp    ┆   ┆ ---   ┆ ---       ┆ ---       ┆ e         │
│ str        ┆ str        ┆ str        ┆ ---       ┆   ┆ u16   ┆ str       ┆ list[str] ┆ ---       │
│            ┆            ┆            ┆ datetime[ ┆   ┆       ┆           ┆           ┆ str       │
│            ┆            ┆            ┆ μs, UTC]  ┆   ┆       ┆           ┆           ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════╪═══════════╪═══════════╪═══════════╡
│ 0194883027 ┆ 002fbe26b2 ┆ {agent_nam ┆ 2025-01-2 ┆ … ┆ 9     ┆ knd_evals ┆ []        ┆ knd_evals │
│ b78ef3cab6 ┆ da8684     ┆ e} run     ┆ 1 09:27:2 ┆   ┆       ┆ run promp ┆           ┆           │
│ 42cf4f9662 ┆            ┆ {prompt=}  ┆ 4.855407  ┆   ┆       ┆ t=Hello,  

In [51]:
result = (
    df_from_arrow
    .with_columns(
        # Count occurrences of "yoo_check" in each list, like counting Dragon Balls
        pl.col("tags")
        .list.eval(lambda x: x.contains("yoo_check"))
        .alias("yoo_check_count")
    )
    .groupby("trace_id")
    .agg(
        pl.sum("yoo_check_count").alias("total_yoo_check")
    )
)


AttributeError: 'Expr' object has no attribute 'apply'